# Úkol č. 4 - regrese (do 2. ledna)

  * Cílem tohoto úkolu je vyzkoušet si řešit regresní problém na reálných (ale celkem vyčištěných) datech.
  
> **Nejdůležitější na úkolu je to, abyste udělali vše procesně správně: korektní rozdělení datasetu, ladění hyperparametrů, vyhodnocení výsledků atp.**

## Dataset

  * Zdrojem dat je list *Data* v souboru `Residential-Building-Data-Set.xlsx` na course pages (originál zde: https://archive.ics.uci.edu/ml/datasets/Residential+Building+Data+Set#).
  * Popis datasetu najdete na listu *Descriptions* ve stejném souboru.
  

## Pokyny k vypracování

  1. Rozdělte data na trénovací a testovací množinu.
  1. Proveďte základní průzkum dat a příp. vyhoďte nezajímavé příznaky.
  1. Aplikujte lineární a hřebenovou regresi a výsledky řádně vyhodnoťte:
    * K měření chyby použijte `mean_absolute_error`.
    * Experimentujte s tvorbou nových příznaků (na základě těch dostupných).
    * Experimentujte se standardizací/normalizací dat.
    * Vyberte si hyperparametry modelů k ladění a najděte jejich nejlepší hodnoty.
  1. Použijte i jiný model než jen lineární a hřebenovou regresi.


## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte pouze tento Jupyter Notebook, opravujíví by neměl nic jiného potřebovat.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

In [80]:
# odtud už je to Vaše
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

Kazdou predikovanou hodnotu budeme predikovat v jednom modelu, tedy budou 2 modely, jeden pro predikci ceny prodeje, ktere nejlepe vystihuje prvnich 8 priznaku, druhy model pro stavebni cenu stavby z nejaktualnejsich dat, tedy z posledniho setu

In [92]:
data = pd.read_excel('Residential-Building-Data-Set.xlsx',list='Data',skiprows=1)

SalePrice_Y=data[['V-9']]       #   predikovana hodnota pro prvni model
CostPrice_Y=data[['V-10']]     #   predikovana hodnota pro druhy model

Nejdrive si vytvorime priznak celkovy pocet mesicu stavby a odstranime nepotrebne priznaky

In [93]:
data['V-30']=(data['COMPLETION YEAR']-data['START YEAR'])*4+(data['COMPLETION QUARTER']-data['START QUARTER'])  #   pocet ctvrtleti, delka doby stavby
#   ponechame pouze rok dokonceni
#   pocet vydanych stavebnich povoleni(V-11) mi prijde zbytecny priznak
to_drop = ['START YEAR','COMPLETION QUARTER','START QUARTER','V-11']

#   data za prubeh 5ti let zprumerujeme, ostatni vyhodime, protoze víme že tato data spolu urcite korelují, jsou na sobe závislé, potřebujeme tyto hodnoty pouze jednou
for i in range(18):
    x=11+i
    row = data[['V-'+str(x),'V-'+str(x)+'.1','V-'+str(x)+'.2','V-'+str(x)+'.3','V-'+str(x)+'.4']]
    data['V-'+str(x)] = row.mean(axis=1)

for i, _ in data.iteritems():
    if '.' in i:
        to_drop.append(i)

data = data.drop(columns=to_drop)
data = data.rename(columns={'COMPLETION YEAR':'V-0'})
display(data)

,V-0,V-1,V-2,V-3,V-4,V-5,V-6,V-7,V-8,V-12,...,V-23,V-24,V-25,V-26,V-27,V-28,V-29,V-9,V-10,V-30
0,85,1,3150.0,920.0,598.5,190,1010.84,16,1200,53.92,...,1755.000,8007.4,64.004,58.608,3402.236,29574.90,628132.9,2200,410,16
1,89,1,7600.0,1140.0,3040.0,400,963.81,23,2900,97.24,...,8647.788,8675.2,98.188,97.958,12544.810,31950.00,1188995.8,5000,1000,23
2,81,1,4800.0,840.0,480.0,100,689.84,15,630,38.96,...,1755.000,6161.8,42.166,34.992,1571.104,27119.60,524764.8,1200,170,15
3,73,1,685.0,202.0,13.7,20,459.54,4,140,11.14,...,1489.818,1528.8,10.458,9.278,522.700,18075.15,141542.6,165,30,4
4,90,1,3000.0,800.0,1230.0,410,631.91,13,5000,176.16,...,9272.228,9343.0,144.290,150.740,9808.600,34438.00,2318397.0,5500,700,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,86,20,1350.0,350.0,108.0,80,251.37,9,830,92.42,...,8548.072,8578.8,94.538,94.006,12233.302,29482.05,1067772.0,1100,150,9
368,82,20,600.0,150.0,36.0,60,299.55,6,570,55.48,...,2988.896,8005.8,66.168,60.886,3665.396,23479.00,648845.6,740,80,6
369,86,20,1900.0,430.0,285.0,150,364.41,7,640,111.38,...,8920.690,8940.2,107.640,108.684,11764.864,30216.90,1181856.2,860,220,7
370,84,20,510.0,160.0,30.6,60,245.28,9,790,69.60,...,8065.908,8118.6,79.512,76.556,6122.494,26394.75,833494.6,1100,110,9


vytvorime si funkce na rozdeleni dat, vizualizace a korelaci dat

Hledame teda priznaky, ktere by mohli ovlivnovat jednotlive predikce. Zkusime si to vyhodnotit podle koleracnich matic. metoda getMAE je pomocna pro rychle vypsani mean_absolute_error, pak mame jednu metodu ktera odstranuje nejmensi prispevky z X-ove mnoziny a druhou, ktera naopak odstranuje dvojice, ktere spolu koleruji natolik, ze by mohli davat zavadejici vysledky

In [94]:
def splitData(x_data,y_data):
    X_train, X_test, Y_train, Y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=42)
    # print('Train X shape: ', X_train.shape)
    # print('Test X shape: ', X_test.shape)
    return X_train, X_test, Y_train, Y_test

def visualization(data,linear):   
    plt.scatter(data, linear)
    plt.plot([0,50], [0,50], 'r')
    plt.show() 

def getMAE(type, x_tr,x_val,y_tr,y_val,opt_params={}):
    if type == 'linear':
        model = LinearRegression(**opt_params)
        model.fit(x_tr,y_tr)
        Yth = model.predict(x_val)
        res = mean_absolute_error(Yth, np.array(y_val))
        print('Linear regression MAE:' + str(res))
        return res
        
    elif type == 'ridge':
        model = Ridge(**opt_params)
        model.fit(x_tr,y_tr)
        Yth = model.predict(x_val)
        res = mean_absolute_error(Yth, np.array(y_val))
        print('Ridge regression MAE:' + str(res))
        return res
    
    else:
        model = RandomForestRegressor(**opt_params)
        model.fit(x_tr,y_tr)
        Yth = model.predict(x_val)
        res = mean_absolute_error(Yth, np.array(y_val))
        print('Random forest regression MAE:' + str(res))
        return res


def featureSelectionMaxKorelation(x_data,y_data):

    #   split data
    x_tr,x_test,y_tr,y_test = splitData(x_data,y_data)
    x_tr,x_val,y_tr,y_val = splitData(x_tr,y_tr)

    p_lr_mae = getMAE('linear',x_tr,x_val,y_tr,y_val) 
    p_rr_mae = getMAE('ridge',x_tr,x_val,y_tr,y_val)

    p_x_data = x_data 
    
    while(True):
        print("-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
        #   najdi nejvice korelovane sloupce
        corrMatrix = x_data.corr().abs()
        corrMatrix =pd.DataFrame(corrMatrix) 
        maxs=dict()
        for col in corrMatrix.columns:
            v = corrMatrix[col].nlargest(2).drop(index=col)
            index = str(np.asarray(v.index))[2:-2]
            a = sorted([col,index])
            maxs[a[0]+','+a[1]] = v[0]
        to_drop = list(max(maxs,key=maxs.get).split(','))
        _ = maxs.pop(max(maxs,key=maxs.get))

        #   odstran nejmensi prispevek
        print("REMOVING --> "+ str(to_drop))
        x_data = x_data.drop(columns=to_drop)

        #   rozdel data
        x_tr,x_test,y_tr,y_test = splitData(x_data,y_data)
        x_tr,x_val,y_tr,y_val = splitData(x_tr,y_tr)

        #   spocitej presnost
        lr_mae = getMAE('linear',x_tr,x_val,y_tr,y_val)
        rr_mae = getMAE('ridge',x_tr,x_val,y_tr,y_val)
        _ = getMAE('random',x_tr,x_val,y_tr,y_val)

        #   jestli je nizsi,uloz a pokracuj || jestli je vyssi, stop, vrat posledni stav
        if(p_lr_mae >= lr_mae and p_rr_mae >= rr_mae):
            p_lr_mae = lr_mae
            p_rr_mae = rr_mae
            p_x_data = x_data.copy()
        else:
            print("--------------------------")
            print("ENDING")
            return p_x_data


def featureSelectionMinKorelation(x_data,y_data,column):

    #   split data
    x_data_nc = x_data.drop(columns=column)
    x_tr,x_test,y_tr,y_test = splitData(x_data_nc,y_data)
    x_tr,x_val,y_tr,y_val = splitData(x_tr,y_tr)

    p_lr_mae = getMAE('linear',x_tr,x_val,y_tr,y_val) 
    p_rr_mae = getMAE('ridge',x_tr,x_val,y_tr,y_val)

    p_x_data = x_data 
    while(True):
        print("-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
        #   najdi nejvice korelovane sloupce
        corrMatrix = x_data.corr().abs()
        corrMatrix =pd.DataFrame(corrMatrix[column]).sort_values(by=column,ascending=True)
        to_drop = corrMatrix.T.columns[0]

        #   odstran nejmensi prispevek
        print("REMOVING --> "+ str(to_drop))
        x_data = x_data.drop(columns=to_drop)
        x_data_nc = x_data.drop(columns=column)

        #   rozdel data
        x_data_nc = x_data.drop(columns=column)
        x_tr,x_test,y_tr,y_test = splitData(x_data_nc,y_data)
        x_tr,x_val,y_tr,y_val = splitData(x_tr,y_tr)

        #   spocitej presnost
        lr_mae = getMAE('linear',x_tr,x_val,y_tr,y_val)
        rr_mae = getMAE('ridge',x_tr,x_val,y_tr,y_val)
        _ = getMAE('random',x_tr,x_val,y_tr,y_val)

        #   jestli je nizsi,uloz a pokracuj || jestli je vyssi, stop, vrat posledni stav
        if(p_lr_mae >= lr_mae and p_rr_mae >= rr_mae):
            p_lr_mae = lr_mae
            p_rr_mae = rr_mae
            p_x_data = x_data.copy()
        else:
            print("--------------------------")
            print("ENDING")
            return p_x_data.drop(columns=column)


Mame 2 predikovane hodnoty, pro kazdou bude existovaj jiny model. Pro toty modely si najdeme nejlepsi mozne priznaky pomoci vyse vytvorenych metod

In [108]:
#   pro predikovanou hodnotu ceny prodeje domu V-9
SalePrice_X = featureSelectionMinKorelation(data.copy(),SalePrice_Y,'V-9')
SalePrice_X = featureSelectionMaxKorelation(SalePrice_X,SalePrice_Y)

print('\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
print("   SALE PRICE BEST SCORE:     ")
n_sp_x_tr,n_sp_x_test,n_sp_y_tr,n_sp_y_test = splitData(SalePrice_X,SalePrice_Y)
x_tr,x_val,y_tr,y_val = splitData(n_sp_x_tr,n_sp_y_tr)
lr_mae = getMAE('linear',x_tr,x_val,y_tr,y_val)
rr_mae = getMAE('ridge',x_tr,x_val,y_tr,y_val)
_ = getMAE('random',x_tr,x_val,y_tr,y_val)
print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n')

# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#    SALE PRICE BEST SCORE:     
# Linear regression MAE:85.14037236077927
# Ridge regression MAE:85.23994359161476
# Random forest regression MAE:135.55714285714282
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

#   pro predikovanou hodnotu ceny stavby donu V-9
CostPrice_X = featureSelectionMinKorelation(data.copy(),CostPrice_Y,'V-10')
CostPrice_X = featureSelectionMaxKorelation(CostPrice_X,CostPrice_Y)

print('\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
print("   COST PRICE BEST SCORE:     ")
n_cp_x_tr,n_cp_x_test,n_cp_y_tr,n_cp_y_test = splitData(CostPrice_X,CostPrice_Y)
x_tr,x_val,y_tr,y_val = splitData(n_cp_x_tr,n_cp_y_tr)
lr_mae = getMAE('linear',x_tr,x_val,y_tr,y_val)
rr_mae = getMAE('ridge',x_tr,x_val,y_tr,y_val)
_ = getMAE('random',x_tr,x_val,y_tr,y_val)
print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n')

# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#    COST PRICE BEST SCORE:     
# Linear regression MAE:20.265471620498555
# Ridge regression MAE:20.113323488298843
# Random forest regression MAE:22.14571428571429
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=



Linear regression MAE:100.42772106161414
Ridge regression MAE:99.44012327277656
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
REMOVING --> V-30
Linear regression MAE:100.42772106157032
Ridge regression MAE:99.42291611580784
Random forest regression MAE:152.9742857142857
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
REMOVING --> V-7
Linear regression MAE:96.99366447365969
Ridge regression MAE:96.78293308607151
Random forest regression MAE:153.6057142857143
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
REMOVING --> V-3
Linear regression MAE:90.36183529348939
Ridge regression MAE:90.24741397349881
Random forest regression MAE:148.53428571428577
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
REMOVING --> V-6
Linear regression MAE:91.85618475525504
Ridge regression MAE:91.85603230965614
Random forest regression MAE:145.4842857142857
--------------------------
ENDING
Linear regression MAE:90.36183529348939
Ridge regression MAE:90.24741397349881
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

Ted zkusíme data standardizovat

In [109]:
scaler = StandardScaler()
scaler.fit(data)
s_data = pd.DataFrame(scaler.transform(data),columns=(data.columns))
# display(s_data)     

A podivat se, jestli se nam presnost modelu zlepsila

In [110]:
#   pro predikovanou hodnotu ceny prodeje domu V-9
s_SalePrice_X = featureSelectionMinKorelation(s_data.copy(),SalePrice_Y,'V-9')
s_SalePrice_X = featureSelectionMaxKorelation(s_SalePrice_X,SalePrice_Y)

print('\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
print("   SALE PRICE BEST SCORE:     ")
s_sp_x_tr,s_sp_x_test,s_sp_y_tr,s_sp_y_test = splitData(s_SalePrice_X,SalePrice_Y)
x_tr,x_val,y_tr,y_val = splitData(s_sp_x_tr,s_sp_y_tr)
lr_mae = getMAE('linear',x_tr,x_val,y_tr,y_val)
rr_mae = getMAE('ridge',x_tr,x_val,y_tr,y_val)
_ = getMAE('random',x_tr,x_val,y_tr,y_val)
print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n')

# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#    SALE PRICE BEST SCORE:     
# Linear regression MAE:100.4277210616299
# Ridge regression MAE:91.32162940110996
# Random forest regression MAE:135.65428571428572
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

#   pro predikovanou hodnotu ceny stavby donu V-9
s_CostPrice_X = featureSelectionMinKorelation(s_data.copy(),CostPrice_Y,'V-10')
s_CostPrice_X = featureSelectionMaxKorelation(s_CostPrice_X,CostPrice_Y)

print('\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
print("   COST PRICE BEST SCORE:     ")
s_cp_x_tr,s_cp_x_test,s_cp_y_tr,s_cp_y_test = splitData(s_CostPrice_X,CostPrice_Y)
x_tr,x_val,y_tr,y_val = splitData(s_cp_x_tr,s_cp_y_tr)
lr_mae = getMAE('linear',x_tr,x_val,y_tr,y_val)
rr_mae = getMAE('ridge',x_tr,x_val,y_tr,y_val)
_ = getMAE('random',x_tr,x_val,y_tr,y_val)
print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n')

# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
#    COST PRICE BEST SCORE:     
# Linear regression MAE:20.26547162050385
# Ridge regression MAE:19.061240126921355
# Random forest regression MAE:22.955714285714286
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


Linear regression MAE:100.4277210616299
Ridge regression MAE:91.32162940110996
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
REMOVING --> V-30
Linear regression MAE:100.42772106163017
Ridge regression MAE:91.33036325724674
Random forest regression MAE:151.38142857142856
--------------------------
ENDING
Linear regression MAE:100.4277210616299
Ridge regression MAE:91.32162940110996
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
REMOVING --> ['V-30', 'V-7']
Linear regression MAE:96.99366447366619
Ridge regression MAE:95.52629235278889
Random forest regression MAE:151.11142857142855
--------------------------
ENDING

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
   SALE PRICE BEST SCORE:     
Linear regression MAE:100.4277210616299
Ridge regression MAE:91.32162940110996
Random forest regression MAE:145.27571428571432
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

Linear regression MAE:21.01043163372758
Ridge regression MAE:19.358119562141688
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-


In [ ]:
Porovnani:
   pred standardizaci                                              po standardizaci
   
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=                     -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
   COST PRICE BEST SCORE:                                       COST PRICE BEST SCORE:     
Linear regression MAE:20.265471620498555                     Linear regression MAE:20.26547162050385
Ridge regression MAE:20.113323488298843                      Ridge regression MAE:19.061240126921355
Random forest regression MAE:22.14571428571429               Random forest regression MAE:22.955714285714286      
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=                     -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=                     -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
   SALE PRICE BEST SCORE:                                       SALE PRICE BEST SCORE:        
Linear regression MAE:85.14037236077927                      Linear regression MAE:100.4277210616299
Ridge regression MAE:85.23994359161476                       Ridge regression MAE:91.32162940110996
Random forest regression MAE:135.55714285714282              Random forest regression MAE:135.65428571428572
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=                     -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

Nyní zkusme iterovat přes množinu různých parametrů a vypsat si jejich skóre

In [128]:
#   linear regresion hyperparametry --> moc jich tam k ladeni neni
param_grid = {
    "fit_intercept" : [True, False],
    "normalize" : [True, False],    #   bezi pokud je fit_intercept == false
    "n_jobs" : [None,1,10,20]
}

cp_lr_gs = GridSearchCV(estimator = LinearRegression(), param_grid = param_grid, cv = 3, verbose=0)
cp_lr_gs.fit(n_cp_x_tr,n_cp_y_tr)
print("optimal parameters for linear regression costPrice: "+ str(cp_lr_gs.best_params_))


sp_lr_gs = GridSearchCV(estimator = LinearRegression(), param_grid = param_grid, cv = 3, verbose=0)
sp_lr_gs.fit(n_sp_x_tr,n_sp_y_tr)
print("optimal parameters for linear regression salePrice: "+ str(sp_lr_gs.best_params_))


#   ridge regression hyperparametry
param_grid = {
    "fit_intercept" : [True, False],
    "normalize" : [True, False],    #   bezi pokud je fit_intercept == false
    "max_iter" : [None,1,10,20],
    "solver": ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga"]
}

cp_rr_gs = GridSearchCV(estimator = Ridge(), param_grid = param_grid, cv = 3, verbose=0)
cp_rr_gs.fit(n_cp_x_tr,n_cp_y_tr)
print("optimal parameters for ridge regression costPrice: "+ str(cp_rr_gs.best_params_))


sp_rr_gs = GridSearchCV(estimator = Ridge(), param_grid = param_grid, cv = 3, verbose=0)
sp_rr_gs.fit(n_sp_x_tr,n_sp_y_tr)
print("optimal parameters for ridge regression salePrice: "+ str(sp_rr_gs.best_params_))

#   random forest hyperparametry
param_grid = {
    "n_estimators" : range(1,100,5),
    "max_features" : ['auto', 'sqrt'],
    "max_depth" : range(1,10)
}

cp_fr_gs = GridSearchCV(estimator = RandomForestRegressor(), param_grid = param_grid, cv = 3, verbose=0)
cp_fr_gs.fit(n_cp_x_tr,n_cp_y_tr)
print("optimal parameters for random forest costPrice: "+ str(cp_fr_gs.best_params_))


sp_fr_gs = GridSearchCV(estimator = RandomForestRegressor(), param_grid = param_grid, cv = 3, verbose=0)
sp_fr_gs.fit(n_sp_x_tr,n_sp_y_tr)
print("optimal parameters for random forest salePrice: "+ str(sp_fr_gs.best_params_))



optimal parameters for linear regression costPrice: {'fit_intercept': True, 'n_jobs': None, 'normalize': False}
optimal parameters for linear regression salePrice: {'fit_intercept': True, 'n_jobs': None, 'normalize': False}
optimal parameters for ridge regression costPrice: {'fit_intercept': True, 'max_iter': None, 'normalize': False, 'solver': 'svd'}
optimal parameters for ridge regression salePrice: {'fit_intercept': True, 'max_iter': None, 'normalize': False, 'solver': 'svd'}
optimal parameters for random forest costPrice: {'max_depth': 7, 'max_features': 'auto', 'n_estimators': 6}
optimal parameters for random forest salePrice: {'max_depth': 7, 'max_features': 'auto', 'n_estimators': 26}


Na závěr vypíšeme všechny MAE pro nestandardizovana (vyslo to lepe) data pro vsechny modely na testovaci mnozine, ktera je zatim nedotcena

In [129]:
print('\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
print("   COST PRICE BEST SCORE:     ")
lr_mae = getMAE('linear',n_cp_x_tr,n_cp_x_test,n_cp_y_tr,n_cp_y_test, cp_lr_gs.best_params_)
rr_mae = getMAE('ridge',n_cp_x_tr,n_cp_x_test,n_cp_y_tr,n_cp_y_test, cp_rr_gs.best_params_)
_ = getMAE('random',n_cp_x_tr,n_cp_x_test,n_cp_y_tr,n_cp_y_test, cp_fr_gs.best_params_)
print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n')

print('\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
print("   SALE PRICE BEST SCORE:     ")
lr_mae = getMAE('linear',n_sp_x_tr,n_sp_x_test,n_sp_y_tr,n_sp_y_test)
rr_mae = getMAE('ridge',n_sp_x_tr,n_sp_x_test,n_sp_y_tr,n_sp_y_test)
_ = getMAE('random',n_sp_x_tr,n_sp_x_test,n_sp_y_tr,n_sp_y_test)
print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n')



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
   COST PRICE BEST SCORE:     
Linear regression MAE:18.492311209794
Ridge regression MAE:18.48144411734697
Random forest regression MAE:21.78052645923296
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
   SALE PRICE BEST SCORE:     
Linear regression MAE:131.8593288342705
Ridge regression MAE:131.84115246612586
Random forest regression MAE:176.44086021505376
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=



Absolutni chyba na testovacich datech je pro predikci SalePrice docela vyrazne vyssi nez na datech testovacich. Naopak u predikce CostPrice vidime i vyssi presnost.